In [2]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd
# 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 기본값으로 설정 (처음 5개와 마지막 5개 행만 출력)
pd.reset_option('display.max_rows')

# 출력 포맷 설정 (소수점 4자리까지)
pd.options.display.float_format = '{:.4f}'.format

import platform
import seaborn as sns

import matplotlib.pyplot as plt

# 운영 체제 확인
if platform.system() == 'Darwin':  # Mac
    print('apple gothic')
    font_name = 'AppleGothic'
elif platform.system() == 'Windows':  # Windows
    font_name = 'NanumGothic'
else:
    font_name = None

# 한글 폰트 설정
if font_name:
    plt.rcParams['font.family'] = font_name

# 마이너스 부호 설정
plt.rcParams['axes.unicode_minus'] = False

apple gothic


## 재무 데이터 로드 및 결합

In [3]:
# (민식) 재무데이터_1.csv : 성장성, 수익성, 자산자본관계비율, 파생변수, 생산성
df1 = pd.read_csv('../data/재무데이터_1.csv', dtype = {'거래소코드' : 'object'}, encoding='cp949')
df1['거래소코드'] = df1['거래소코드'].str.zfill(6)

# (지현) 재무데이터2.csv
df2 = pd.read_csv('../data/재무데이터_2.csv', dtype = {'거래소코드' : 'object'}, encoding='cp949')
df2['거래소코드'] = df2['거래소코드'].str.zfill(6)

# 불필요 컬럼 제거
df1.drop(columns = ['개월', '매출액(수익)', '비유동부채 ', '회사'], inplace=True)
df2.drop(columns = ['개월'], inplace=True)

In [4]:
df1.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '유형자산', '유동자산', '재고자산', '자본', '매출액',
       '매출원가', '판매비와 관리비(물류원가 등 포함)', '기타(영업)비용', '당기제조원가', '급여', '퇴직급여',
       '상품매출원가', '제품매출원가', '감가상각비', '대손상각비', '개발비상각', '기타무형자산상각비', '외환차익1',
       '외화환산이익1', '외환차손1', '외화환산손실1', '외환차익2', '외화환산이익2', '외환차손2', '외환환산손실2',
       '무형자산', '건설중인자산', '토지', '자본잉여금', '이익잉여금(결손금)', '미처분이익잉여금(결손금)',
       '기타임의적립금', '(연차배당)', '(중간배당)', '배당금지급(-)', '단기차입금', '매입채무 및 기타유동채무',
       '매출채권 및 기타유동채권', '비유동자산', '사채', '유동금융자산', '유동부채', '유동성장기부채',
       '장기매입채무 및 기타비유동채무', '장기매출채권 및 기타비유동채권', '장기차입금', '현금및현금성자산',
       '영업활동으로 인한 현금흐름', '유형자산의 증가', '무형자산의 증가', '연구개발비',
       '종업원1인당 매출액(IFRS)(백만원)', '종업원1인당 부가가치(IFRS)(백만원)',
       '종업원1인당 인건비(IFRS)(백만원)', '종업원수(IFRS)', '판매비와 관리비'],
      dtype='object')

In [3]:
# 회사명, 거래소코드, 회계년도 기준 재무데이터 결합

df_fin = pd.merge(
    df1, df2,
    on = ['회사명', '거래소코드', '회계년도'],
    suffixes = ('', '_right')
)

df_fin.shape, df1.shape, df2.shape

((26351, 87), (26351, 60), (26543, 30))

In [4]:
# 중복 컬럼 확인
drop_cols = df_fin.columns[df_fin.columns.str.endswith('_right')]
len(drop_cols), drop_cols

(18,
 Index(['자본_right', '자산_right', '유동자산_right', '유동부채_right', '현금및현금성자산_right',
        '유동금융자산_right', '매출채권 및 기타유동채권_right', '비유동자산_right', '장기차입금_right',
        '단기차입금_right', '사채_right', '장기매출채권 및 기타비유동채권_right',
        '장기매입채무 및 기타비유동채무_right', '매입채무 및 기타유동채무_right', '매출액_right',
        '건설중인자산_right', '유형자산_right', '재고자산_right'],
       dtype='object'))

In [5]:
# 중복 컬럼 제거
df_fin.drop(columns = drop_cols, inplace=True)

# 중복 컬럼 제거 후 컬럼 수
len(df_fin.columns), df_fin.columns

(69,
 Index(['회사명', '거래소코드', '회계년도', '자산', '유형자산', '유동자산', '재고자산', '자본', '매출액',
        '매출원가', '판매비와 관리비(물류원가 등 포함)', '기타(영업)비용', '당기제조원가', '급여', '퇴직급여',
        '상품매출원가', '제품매출원가', '감가상각비', '대손상각비', '개발비상각', '기타무형자산상각비', '외환차익1',
        '외화환산이익1', '외환차손1', '외화환산손실1', '외환차익2', '외화환산이익2', '외환차손2', '외환환산손실2',
        '무형자산', '건설중인자산', '토지', '자본잉여금', '이익잉여금(결손금)', '미처분이익잉여금(결손금)',
        '기타임의적립금', '(연차배당)', '(중간배당)', '배당금지급(-)', '단기차입금', '매입채무 및 기타유동채무',
        '매출채권 및 기타유동채권', '비유동자산', '사채', '유동금융자산', '유동부채', '유동성장기부채',
        '장기매입채무 및 기타비유동채무', '장기매출채권 및 기타비유동채권', '장기차입금', '현금및현금성자산',
        '영업활동으로 인한 현금흐름', '유형자산의 증가', '무형자산의 증가', '연구개발비',
        '종업원1인당 매출액(IFRS)(백만원)', '종업원1인당 부가가치(IFRS)(백만원)',
        '종업원1인당 인건비(IFRS)(백만원)', '종업원수(IFRS)', '판매비와 관리비', '비유동부채', '영업활동현금흐름',
        '부채', '유형자산의증가', '유형자산의감소', '(투자활동으로 인한 현금유출액)', '투자활동으로 인한 현금유입액',
        '자본금', '관계기업 등 지분관련 투자자산'],
       dtype='object'))

## 이자보상배율 데이터 로드

In [6]:
df = pd.read_csv('../data/1_불필요종목제거.csv', dtype = {'거래소코드' : 'object'})
df.shape

(22245, 20)

In [7]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '[제조]이자보상배율(이자비용)(IFRS연결)',
       '[제조]이자보상배율(이자비용)(IFRS)',
       '[U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS연결)(천원)',
       '[U01B470010000][제조]   이자비용(IFRS연결)(천원)',
       '[U01B550010000][제조]   이자비용(IFRS연결)(천원)',
       '[U01B201013300][제조]   이자비용(IFRS연결)(천원)',
       '[U01B350016400][제조]   이자비용(IFRS연결)(천원)',
       '[U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS)(천원)',
       '[U01B470010000][제조]   이자비용(IFRS)(천원)',
       '[U01B550010000][제조]   이자비용(IFRS)(천원)',
       '[U01B201013300][제조]   이자비용(IFRS)(천원)',
       '[U01B350016400][제조]   이자비용(IFRS)(천원)', '결산년도', '상장년도', 'market'],
      dtype='object')

## 이자보상배율+재무데이터 결합

In [8]:
# 회사명, 거래소코드, 회계년도 기준 재무데이터 결합

df_merged = pd.merge(
    df, df_fin,
    on = ['회사명', '거래소코드', '회계년도'],
    suffixes = ('', '_right'),
    how = 'left'
)

df_merged.shape, df.shape, df_fin.shape

((22245, 86), (22245, 20), (26351, 69))

In [9]:
# 중복 컬럼 확인
drop_cols = df_merged.columns[df_merged.columns.str.endswith('_right')]
len(drop_cols), drop_cols

(0, Index([], dtype='object'))

## 파일로 저장

In [10]:
df_merged.to_csv('../data/2_재무데이터결합.csv', index=None)